In [0]:
# Build Bronze: raw ingestion

bronze_df = spark.read.csv("/Volumes/workspace/ecommerce/ecommerce_data/2019-Oct.csv", header=True, inferSchema=True)

In [0]:
bronze_df.write.format("delta").mode("overwrite").saveAsTable("ecommerce.bronze_table")


In [0]:
%sql
select * FROM ecommerce.bronze_table

In [0]:
%sql
--  Build Silver: cleaning & validation 


create or replace table ecommerce.silver_table
using delta
as
select distinct event_time, event_type, product_id,category_id,category_code, brand, price, user_id, user_session, price_tier
from (select *,
  case 
    when price<10 then "Budget"
    when price<50 then "Mid"
    else "Premium"
  end as Price_tier
from ecommerce.bronze_table)
where user_id is not null
or product_id is not null
or price>0;


In [0]:
%sql
-- Build Gold: business aggregates

CREATE OR REPLACE TABLE ecommerce.gold_product_sales
USING DELTA
AS
SELECT product_id,category_code, brand,
COUNT(*) AS total_purchases,
ROUND(SUM(price), 2) AS total_sales
FROM ecommerce.silver_table
WHERE event_type = 'purchase'
GROUP BY product_id, category_code, brand
ORDER BY total_sales DESC;

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold_event_summary
USING DELTA
AS
SELECT event_type,count(event_type) as total_events
FROM ecommerce.silver_table
GROUP BY event_type;
select * from ecommerce.gold_event_summary;

In [0]:
%sql
CREATE OR REPLACE TABLE ecommerce.gold_user_activity
USING DELTA
AS
SELECT user_id, COUNT(*) AS total_events, COUNT(CASE WHEN event_type = 'view' THEN 1 END) AS total_views,
COUNT(CASE WHEN event_type = 'cart' THEN 1 END) AS total_cart_adds,
COUNT(CASE WHEN event_type = 'purchase' THEN 1 END) AS total_purchases,
SUM(CASE WHEN event_type = 'purchase' THEN price ELSE 0 END) AS total_spent
FROM ecommerce.silver_table
GROUP BY user_id;